In [1]:
import sys
sys.path.append("../")
import nanonis
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import xlrd
import lmfit
from lmfit import Model
import matplotlib.colors as colors
from scipy import constants as const
%matplotlib qt
spectra=nanonis.biasSpectroscopy()

# File management #

In [23]:
data=pd.read_excel("C:/Users/jonor/Documents/GitHub/nanonis/Order.xls")

In [33]:
Bfield_files=data.iloc[:,2]
B=data.iloc[:,3]
size_dat=data.iloc[:,7]
height_dat=data.iloc[:,8]
temp=data.iloc[:,6]
temp_files=data.iloc[:,5]
josephson=data.iloc[:,4]
path=data.iloc[:,10]

In [35]:
dat=[ 8, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 23, 24]
T=[]
Tfiles=[]
Bfiles=[]
Jfiles=[]
size=[]
height=[]
Bfiles=[]
B=[]
for j in dat:
    T.append([float(i) for i in temp[j].split(',')])
    Tfiles.append([path[j]+i for i in temp_files[j].split(',')])
    size.append(size_dat[j])
    height.append(height_dat[j])
    B.append([int(i) for i in Bfields[j].split(',')])
    Bfiles.append([path[j]+i for i in Bfield_files[j].split(',')])
    Jfiles.append([path[j]+i for i in josephson[j].split(',')])

# Functions

In [100]:
def fermi(T,x):
    return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1(V,R1,C1,C2,n,Q0,T):
    k=C2/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    x=E1(C1,C2,V,n,Q0)
    return (x/(1-np.exp(-x/T)))/R1

def Gamma2(V,R2,C1,C2,n,Q0,T):
    k=C1/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    x=E2(C1,C2,V,-n,-Q0)
    return (x/(1-np.exp(-x/T)))/R2


def PN(V,R1,R2,C1,C2,Q0,T,n):
    mn=(Gamma1(V,R1,C1,C2,-n,Q0,T)+Gamma2(-V,R2,C1,C2,n,-Q0,T))/(Gamma1(-V,R1,C1,C2,n+1,-Q0,T)+Gamma2(V,R2,C1,C2,-n-1,Q0,T))
    en=(Gamma1(-V,R1,C1,C2,-n,-Q0,T)+Gamma2(V,R2,C1,C2,n,Q0,T))/(Gamma1(V,R1,C1,C2,n+1,Q0,T)+Gamma2(-V,R2,C1,C2,-n-1,-Q0,T))
    return en,mn

def check_p(V,R1,R2,C1,C2,Q0,T):
    Vmax=V
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PN(Vmax,R1,R2,C1,C2,Q0,T,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if np.sum(pn)<0.001 and np.sum(p_n)<0.001:
            #print(len(a))
            break 
    return n[-1]

def all_p(V,R1,R2,C1,C2,Q0,T,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PN(V,R1,R2,C1,C2,Q0,T,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp=np.zeros(n)
        temp[0:i+1]=1
        temp=temp.tolist()
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(n):
            if temp[j]<=0.1:
                pass
            else:
                temp1*=a[j]       
                temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1n(V,R1,C1,C2,Q0,T,n):
    return Gamma1(V,R1,C1,C2,n,Q0,T)-Gamma1(-V,R1,C1,C2,-n,-Q0,T)

def G2n(V,R2,C1,C2,Q0,T,n):
    return Gamma2(V,R2,C1,C2,n,Q0,T)-Gamma2(-V,R2,C1,C2,-n,-Q0,T)

def current(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G1n(V,R1,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1n(V,R1,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G1n(V,R1,C1,C2,Q0,T,-i)
    return I/np.abs(I[0])

def conductance(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G1n(V,R1,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1n(V,R1,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G1n(V,R1,C1,C2,Q0,T,-i)
    I=-np.gradient(I)
    return I/np.abs(I[0])


def current2(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G2n(V,R2,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2n(V,R2,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G2n(V,R2,C1,C2,Q0,T,-i)
    I=-np.gradient(I)
    return I/np.abs(I[0])

def gap_size(x,y,epsilon):
    xx=[]
    for i in range(len(y)):
        if np.abs(y[i])<epsilon:
            xx.append(x[i])
    return np.abs(xx[0]-xx[-1])

# Analysis for the B field data

## 1st method: fitting the current

In [ ]:
for i in range(0,13):
    print(i)
    h=0
    cutt=np.arange(10,200,20)
    B800=B[i].index(800)
    R=[]
    R2=[]
    for cut in cutt:
        C1=[]
        Q0=[]
        erC=[]
        erQ0=[]
        for j in Bfiles[i][B800:-1]:
            spectra.load(j)
            x=np.array(spectra.bias)[200-cut:200+cut]
            y=np.array(spectra.current)[200-cut:200+cut]
            y=y/np.abs(y[0])
            #plt.plot(1000*x,y+h,linewidth=2)
            par,error=sc.optimize.curve_fit(lambda xx, C1,Q0: current(xx, 1,10000, C1,0.001,Q0,0.1), 1000*x, y,p0=[0.5,0.0],bounds=([0.01,-0.5],[15.0,0.5]))
            #a=current2(1000*x,1,10000,par[0],0.0001,par[1],0.1)
            #plt.plot(1000*x,a+h)
            #h+=1
            #C1.append(par[0])
            #Q0.append(par[1])
            erC.append(np.sqrt(np.diag(error))[0])
            erQ0.append(np.sqrt(np.diag(error))[1])
        #plt.plot(1/np.array(C1))
        R.append(np.mean(erC))
        R2.append(np.mean(erQ0))
        #plt.plot(erQ0)
    plt.plot(cutt,R)


cut>100 doe snot change much the error

## 2nd method: fitting the conductance

In [106]:
for i in range(0,13):
    print(i)
    h=0
    cutt=np.arange(10,200,20)
    B800=B[i].index(800)
    R=[]
    R2=[]
    for cut in cutt:
        C1=[]
        Q0=[]
        erC=[]
        erQ0=[]
        for j in Bfiles[i][B800:-1]:
            spectra.load(j)
            x=np.array(spectra.bias)[200-cut:200+cut]
            y=np.array(spectra.conductance)[200-cut:200+cut]
            y=y/np.abs(y[0])
            #plt.plot(1000*x,y+1*h,linewidth=2)
            par,error=sc.optimize.curve_fit(lambda xx, C1,Q0: conductance(xx, 1,10000, C1,0.001,Q0,0.1), 1000*x, y,p0=[0.5,0.0],bounds=([0.01,-0.5],[2.0,0.5]))
            #a=current2(1000*x,1,10000,par[0],0.0001,par[1],0.1)
            #plt.plot(1000*x,a+1*h)
            h+=1
            C1.append(par[0])
            Q0.append(par[1])
            erC.append(np.sqrt(np.diag(error))[0])
            erQ0.append(np.sqrt(np.diag(error))[1])
        R.append(np.mean(erC))
        R2.append(np.mean(erQ0))
    plt.plot(cutt,R)


0


C:\Users\jonor\AppData\Local\Temp/ipykernel_22396/3435372294.py:16: RuntimeWarning: overflow encountered in exp
  return (x/(1-np.exp(-x/T)))/R1
C:\Users\jonor\AppData\Local\Temp/ipykernel_22396/3435372294.py:22: RuntimeWarning: overflow encountered in exp
  return (x/(1-np.exp(-x/T)))/R2


1
2


C:\Users\jonor\AppData\Local\Temp/ipykernel_22396/3435372294.py:16: RuntimeWarning: overflow encountered in exp
  return (x/(1-np.exp(-x/T)))/R1


3
4
5


RuntimeError: Optimal parameters not found: The maximum number of function evaluations is exceeded.

## 3rd method: calculate the zero current

In [ ]:
for i in range(0,13):
    print(i)

    B800=B[i].index(800)
    Eg=[]
    Q0=[]
    cut=np.arange(10,500)
    for epsilon in 
        for j in Bfiles[i][B800:-1]:
            spectra.load(j)
            x=np.array(spectra.bias)
            y=np.array(spectra.current)
            y=y/np.abs(y[0])

            C1.append(par[0])
            Q0.append(par[1])
            erC.append(np.sqrt(np.diag(error))[0])
            erQ0.append(np.sqrt(np.diag(error))[1])
        R.append(np.mean(erC))
        R2.append(np.mean(erQ0))
plt.plot(cutt,R)
